# US Churn &mdash; Import

## Setup

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, Markdown
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)  

import sys, os, yaml

DATASET = "US_Churn"

ROOT = "./"
COLAB = 'google.colab' in sys.modules
if COLAB:
    ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
    

DEBUG = False
SEED = 1612

In [16]:
if COLAB:
    from google.colab import drive
    if not os.path.isdir("/content/gdrive"):
        drive.mount("/content/gdrive")
        d = "/content/gdrive/MyDrive/datasets"
        if not os.path.isdir(d): os.makedirs(d)
        if not os.path.isdir(ROOT): os.makedirs(ROOT)

def makedirs(d):
    if COLAB:
        if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
    else:
        if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['orig','data','output']: makedirs(d)

## Datasets

In [17]:
BASE_URL = "https://SETU-DataMining2.github.io/live/resources/us_churn"

for filename in ['churn.csv',]:
    source = f"{BASE_URL}/{filename}"
    target = f"{ROOT}/orig/{filename}"

    if not os.path.isfile(target):
        print (f"Downloading remote file {filename}", sep="")
        import urllib.request
        urllib.request.urlretrieve(source, target)
    else:
        print(f"Using local copy of {filename}")

Using local copy of churn.csv


In [18]:
df = pd.read_csv(f"{ROOT}/orig/churn.csv")
print(df.shape)
df.head()

(3333, 21)


State  Account Length  Area Code     Phone Int'l Plan VMail Plan  \
0    KS             128        415  382-4657         no        yes   
1    OH             107        415  371-7191         no        yes   
2    NJ             137        415  358-1921         no         no   
3    OH              84        408  375-9999        yes         no   
4    OK              75        415  330-6626        yes         no   

   VMail Message  Day Mins  Day Calls  Day Charge  Eve Mins  Eve Calls  \
0             25     265.1        110       45.07     197.4         99   
1             26     161.6        123       27.47     195.5        103   
2              0     243.4        114       41.38     121.2        110   
3              0     299.4         71       50.90      61.9         88   
4              0     166.7        113       28.34     148.3        122   

   Eve Charge  Night Mins  Night Calls  Night Charge  Intl Mins  Intl Calls  \
0       16.78       244.7           91         11.01       10.0           3   
1       16.62       254.4          103         11.45       13.7           3   
2       10.30       162.6          104          7.32       12.2           5   
3        5.26       196.9           89          8.86        6.6           7   
4       12.61       186.9          121          8.41       10.1           3   

   Intl Charge  CustServ Calls  Churn?  
0         2.70               1  False.  
1         3.70               1  False.  
2         3.29               0  False.  
3         1.78               2  False.  
4         2.73               3  False.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   State           3333 non-null   object 
 1   Account Length  3333 non-null   int64  
 2   Area Code       3333 non-null   int64  
 3   Phone           3333 non-null   object 
 4   Int'l Plan      3333 non-null   object 
 5   VMail Plan      3333 non-null   object 
 6   VMail Message   3333 non-null   int64  
 7   Day Mins        3333 non-null   float64
 8   Day Calls       3333 non-null   int64  
 9   Day Charge      3333 non-null   float64
 10  Eve Mins        3333 non-null   float64
 11  Eve Calls       3333 non-null   int64  
 12  Eve Charge      3333 non-null   float64
 13  Night Mins      3333 non-null   float64
 14  Night Calls     3333 non-null   int64  
 15  Night Charge    3333 non-null   float64
 16  Intl Mins       3333 non-null   float64
 17  Intl Calls      3333 non-null   i

Issues are:

 * Poor column names - spaces and punctation
 * Inconsistent labels for boolean columns - convert to categorical also
 * Need to encode target column  - convert to categorical also
 * `Area_Code` should be categorical (3 levels)
 * `State` should be categorical (51 levels) <- what should I do here? wait till EDA
 * Unique identifier column `Phone`

In [20]:
# Poor column names - spaces and punctuation
df.columns = [c.replace(" ", "_").replace("'", "").replace("?", "") for c in df.columns]

In [21]:
# Inconsistent labels for boolean columns
for c in [c for c in df.columns if "Plan" in c]:
    if df[c].dtype =="object":
        df[c] = df[c].map( {"no":0, "yes":1} )
        df[c] = pd.Categorical(df[c])

In [22]:
# Need to encode target column
if df.Churn.dtype == "object":
    df.Churn = df.Churn.map( {"False.":0, "True.":1} )
    df.Churn = pd.Categorical(df.Churn)

In [23]:
# Encode Area_Code column
if 0 not in df.Area_Code.unique():
    df.Area_Code = df.Area_Code.map( {415:0, 510:1,408:2 } )
    df.Area_Code = pd.Categorical(df.Area_Code)

In [24]:
if df.State.dtype == "object":
    df.State = pd.Categorical(df.State)
df.State.nunique()

51

In [25]:
# Remove unwanted columns
for c in ["Phone"]: 
    if c in df.columns: df.drop(c, axis='columns',inplace=True)

## Save dataset

In [26]:
df.head()

State  Account_Length Area_Code Intl_Plan VMail_Plan  VMail_Message  \
0    KS             128         0         0          1             25   
1    OH             107         0         0          1             26   
2    NJ             137         0         0          0              0   
3    OH              84         2         1          0              0   
4    OK              75         0         1          0              0   

   Day_Mins  Day_Calls  Day_Charge  Eve_Mins  Eve_Calls  Eve_Charge  \
0     265.1        110       45.07     197.4         99       16.78   
1     161.6        123       27.47     195.5        103       16.62   
2     243.4        114       41.38     121.2        110       10.30   
3     299.4         71       50.90      61.9         88        5.26   
4     166.7        113       28.34     148.3        122       12.61   

   Night_Mins  Night_Calls  Night_Charge  Intl_Mins  Intl_Calls  Intl_Charge  \
0       244.7           91         11.01       10.0           3         2.70   
1       254.4          103         11.45       13.7           3         3.70   
2       162.6          104          7.32       12.2           5         3.29   
3       196.9           89          8.86        6.6           7         1.78   
4       186.9          121          8.41       10.1           3         2.73   

   CustServ_Calls Churn  
0               1     0  
1               1     0  
2               0     0  
3               2     0  
4               3     0

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   State           3333 non-null   category
 1   Account_Length  3333 non-null   int64   
 2   Area_Code       3333 non-null   category
 3   Intl_Plan       3333 non-null   category
 4   VMail_Plan      3333 non-null   category
 5   VMail_Message   3333 non-null   int64   
 6   Day_Mins        3333 non-null   float64 
 7   Day_Calls       3333 non-null   int64   
 8   Day_Charge      3333 non-null   float64 
 9   Eve_Mins        3333 non-null   float64 
 10  Eve_Calls       3333 non-null   int64   
 11  Eve_Charge      3333 non-null   float64 
 12  Night_Mins      3333 non-null   float64 
 13  Night_Calls     3333 non-null   int64   
 14  Night_Charge    3333 non-null   float64 
 15  Intl_Mins       3333 non-null   float64 
 16  Intl_Calls      3333 non-null   int64   
 17  Intl_Charge   

In [28]:
df.to_pickle(f"{ROOT}/data/churn.pkl")